In [1]:
from my_trader import *

C:\ProgramData\Anaconda2\lib\site-packages\fix_yahoo_finance\__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and no longer available.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [2]:
robinhood = get_robinhood()
stock1 = "VIXY"
stock2 = "SPY"
window = 30
data_len = 210
initial = 1500


price1 = get_price_data([stock1],method="day",start_date=datetime.now()-timedelta(days=data_len),end_date=datetime.now()-timedelta(days=0))
price2 = get_price_data([stock2],method="day",start_date=datetime.now()-timedelta(days=data_len),end_date=datetime.now()-timedelta(days=0))
#price1 = get_price_data([stock1],method = "robinhood")
#price2 = get_price_data([stock2],method = "robinhood")
price1 = price1.set_index("TimeStamp")
price2 = price2.set_index("TimeStamp")
price1 = price1.rename(columns={"Close":stock1+"_close"})
price2 = price2.rename(columns={"Close":stock2+"_close"})



price_table = pd.concat([price1[stock1+"_close"],price2[stock2+"_close"]],axis = 1)
price_table = price_table.fillna(price_table.shift(1))
price_table[stock1+"_close"] = price_table[stock1+"_close"].astype(float)
price_table[stock2+"_close"] = price_table[stock2+"_close"].astype(float)
price_table[stock1+"_log_ret"] = log(price_table[stock1+"_close"] / price_table[stock1+"_close"].shift(1))
price_table[stock2+"_log_ret"] = log(price_table[stock2+"_close"] / price_table[stock2+"_close"].shift(1))

#price_table=price_table.fillna(1)

price_table[stock1+"_log_ret_mv"] = price_table[stock1+"_log_ret"].rolling(window).mean()
price_table[stock2+"_log_ret_mv"] =price_table[stock2+"_log_ret"].rolling(window).mean()
price_table["relative"]=price_table[stock1+"_log_ret"]/price_table[stock2+"_log_ret"]
price_table["relative_mv"] = price_table["relative"].rolling(window).mean()

price_table.relative.loc[price_table.relative==-np.inf]=np.NaN

#price_table["relative_mv"]=price_table[stock1+"_log_ret_mv"]/price_table[stock2+"_log_ret_mv"]
price_table["z_score"] =( price_table["relative"]-price_table["relative_mv"])/price_table.relative.std()
price_table["trade"]=0
price_table["trade_signal"]=np.NaN
price_table["slope"]=0
for i in range(window-1,len(price_table)):
    price_table["slope"].iloc[i] = stats.linregress(price_table[stock1+"_log_ret"]\
                                                    .iloc[i-(window-1):i],price_table[stock2+"_log_ret"].iloc[i-(window-1):i])[0]
price_table["buy_line"] = np.NaN



for i in range(window-1,len(price_table)):
    if price_table.slope.iloc[i]>0.5:
        price_table["buy_line"].iloc[i] = -1.25
    elif price_table.slope.iloc[i]>0.75:
        price_table["buy_line"].iloc[i]= -1.75
    elif price_table.slope.iloc[i]<-0.5:
        price_table["buy_line"].iloc[i] -2.25
    elif price_table.slope.iloc[i]<-0.75:
        price_table["buy_line"].iloc[i]= -2.75
    else:
        price_table["buy_line"].iloc[i] = -2
price_table["sell_line"] = np.NaN
for i in range(window-1,len(price_table)):
    if price_table.slope.iloc[i]>0.5:
        price_table["sell_line"].iloc[i] = 1.25
    elif price_table.slope.iloc[i]>0.75:
        price_table["sell_line"].iloc[i] = 1.75
    elif price_table.slope.iloc[i]<-0.5:
        price_table["sell_line"].iloc[i] = 2.25
    elif price_table.slope.iloc[i]<-0.75:
        price_table["sell_line"].iloc[i] = 2.75
    else:
        price_table["sell_line"].iloc[i] = 2  
           
#no short sell

price_table["relative_mv"] = abs(price_table["relative_mv"])

        
#price_table["cost_per_trade"] =abs( price_table[stock1+"_close"]+price_table[stock2+"_close"]*price_table["relative_mv"])
price_table[stock1+"_suggest_shares"] = np.ceil((initial/(1+price_table["relative_mv"])/price_table[stock1+"_close"]))
price_table[stock2+"_suggest_shares"] = np.ceil((initial/(1+price_table["relative_mv"])*price_table["relative_mv"]/price_table[stock2+"_close"]))



#set live trade signal and backtest 
price_table[stock1+"_shares"] = 0 
price_table[stock2+"_shares"] = 0
for i in range(window-1,len(price_table)):
    if price_table.z_score.iloc[i] < price_table.buy_line.iloc[i]:
        price_table["trade"].iloc[i] = 1
        price_table["trade_signal"].iloc[i] = 1
        price_table[stock1+"_shares"].iloc[i] = price_table[stock1+"_suggest_shares"].iloc[i]
        price_table[stock2+"_shares"].iloc[i] = price_table[stock2+"_suggest_shares"].iloc[i]
    elif price_table.z_score.iloc[i] > price_table.sell_line.iloc[i]:
        price_table["trade"].iloc[i] = 0
        price_table["trade_signal"].iloc[i] = 0
        price_table[stock1+"_shares"].iloc[i] = 0 
        price_table[stock2+"_shares"].iloc[i] = 0
    else:
        price_table["trade"].iloc[i] = price_table.trade.iloc[i-1]
        price_table[stock1+"_shares"].iloc[i] = price_table[stock1+"_shares"].iloc[i-1]
        price_table[stock2+"_shares"].iloc[i] = price_table[stock2+"_shares"].iloc[i-1]

#price_table[stock1+"_value"] = price_table[stock1+"_suggest_shares"]*price_table[stock1+"_close"]
#price_table[stock2+"_value"] = price_table[stock2+"_suggest_shares"]*price_table[stock2+"_close"]

price_table[stock1+"_value"] = price_table[stock1+"_shares"]*price_table[stock1+"_close"]
price_table[stock2+"_value"] = price_table[stock2+"_shares"]*price_table[stock2+"_close"]



price_table["p_L"] = price_table[stock1+"_value"].shift(1) * price_table[stock1+"_log_ret"].shift(1)*price_table.trade.shift(1) +  \
price_table[stock2+"_value"].shift(1) * price_table[stock2+"_log_ret"].shift(1)*price_table.trade.shift(1)
price_table["rolling_p_L"] = price_table.p_L.rolling(30).sum()

Finished VIXY
Finished SPY


C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [101]:
price_table.iloc[-1]

VIXY_close               35.470001
SPY_close               269.130005
VIXY_log_ret             -0.032999
SPY_log_ret               0.019511
VIXY_log_ret_mv           0.014600
SPY_log_ret_mv            0.000193
relative                 -1.691329
relative_mv               4.130633
z_score                   0.039613
trade                     1.000000
trade_signal                   NaN
slope                    -0.153826
buy_line                 -2.000000
sell_line                 2.000000
VIXY_suggest_shares       9.000000
SPY_suggest_shares        5.000000
VIXY_shares               2.000000
SPY_shares                6.000000
VIXY_value               70.940002
SPY_value              1614.780030
p_L                     -46.066290
rolling_p_L              14.774481
Name: 2018-02-06 00:00:00, dtype: float64

In [33]:
price_table.to_csv("price_table.csv")

In [37]:
beta(["VIXY"])

(          Beta
 VIXY -0.883393, array([[  6.25742912e-05,  -3.31949627e-04],
        [ -3.31949627e-04,   2.25652577e-03]]), [0.047502902712985122])

In [44]:
robinhood.my_trader.quote_data("VIXY")

{u'adjusted_previous_close': u'27.3100',
 u'ask_price': u'47.0000',
 u'ask_size': 500,
 u'bid_price': u'40.5000',
 u'bid_size': 3000,
 u'has_traded': True,
 u'instrument': u'https://api.robinhood.com/instruments/869251ff-fb17-40c6-9d24-9ab7636dd3d6/',
 u'last_extended_hours_trade_price': u'44.5100',
 u'last_trade_price': u'36.6600',
 u'last_trade_price_source': u'consolidated',
 u'previous_close': u'27.3100',
 u'previous_close_date': u'2018-02-02',
 u'symbol': u'VIXY',
 u'trading_halted': False,
 u'updated_at': u'2018-02-06T01:00:00Z'}

In [23]:
mean_reversion("VIXY","minute")

***********************************************
**********VIXY*******************
Augmented Dickey Fuller test statistic = -1.17422487337
Augmented Dickey Fuller p-value = 0.68458410199
Augmented Dickey Fuller 1%, 5% and 10% test statistics = {'5%': -2.8770397560752436, '1%': -3.4656203971241921, '10%': -2.5750324547306476}
Hurst Exponent = 0.45
Halflife =  126.0


['VIXY', 'Augmented Dickey Fuller: Not significant', 'mean reverting', 'Halflife: 126.0 10_minute']
***********************************************


['VIXY',
 'Augmented Dickey Fuller: Not significant',
 'mean reverting',
 'Halflife: 126.0 10_minute']

In [84]:
price_table.relative.loc[price_table.relative==-np.inf]=np.NaN


In [86]:
price_table.relative.std()

32.365204234006